# Import all necessary modules

In [5]:
# import all neceassary modules
import sys
from Experimentation.monte_carlo import *
from Experimentation.ensemble_models import *
from Experimentation.get_alpha import *
from Experimentation.quantile import *
from Experimentation.weighting_scheme import *
from Experimentation.wis_score import *
from Experimentation.model_averaging import *
import numpy as np
import pandas as pd
import random
random.seed(42)
np.random.seed(42)

In [6]:
filename = 'data/alpha.txt'
alphas = get_alpha(filename)

state = 'New York' #if use Monroe or SantaRosa for Monroe or Santa Rosa county

if state=='New York':
    s_id = '36'
elif state=='Monroe':
    s_id='Monroe'
elif state=='Santa Rosa':
    s_id='SantaRosa'

# Ensemble Weights Calculation

Weights are calculated based on the formula used in paper.

In [4]:

fname = 'data/all_methods_state_{}.csv'.format(51)
data = pd.read_csv(fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
# data = data[data.forecast_date<pd.to_datetime('2021-01-01')]

# phases = pd.read_csv('NY_phts_ind_2022-01-15.csv', parse_dates=['target_end_date','V1'])
# phases['V1'] = phases['V1'] + pd.Timedelta(days=1) 

data = model_weights(data, t_p=4)
data = change_weights_format(data)
data.to_csv('weights/weights_'+fname.split('/')[-1]) ## creates a new file containing weights

KeyboardInterrupt: 

# Ensemble Forecast using LOP, Vincent, Mean, Median averaging

In [13]:
fname = 'weights/phase_weights_all_methods_state_{}.csv'.format(s_id)
weights = pd.read_csv(fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
f_fname = 'all_methods_state_{}.csv'.format(s_id)
data = pd.read_csv(f_fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
out = Vincent_Ensemble(data, weights, alphas)  ## for vincent use Vincent_Ensemble, for mean use Mean_Ensemble, and for median use Median_Ensemble
out.to_csv('Vincent-Ensemble_{}.csv'.format(s_id))

# Calculate WIS

In [20]:
import glob
for file in glob.glob('Vincent-Ensemble_*.csv'):
    
    # fname = 'Vincent-Ensemble_{}_phases.csv'.format(s_id) #filename to get WIS score
    fname = file.split('/')[-1]
    data = pd.read_csv(file, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
    data = get_all_wis_score(data, alphas)
    data.to_csv(file)

In [5]:
###get median
for file in glob.glob('wis-ensemble/*'):
    fname = file.split('/')[-1].split('.')[0]
# fname = 'Vincent-Ensemble_{}.csv'.format(s_id) #filename to get WIS score
    data = pd.read_csv(file, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
    data = get_all_wis_score(data, alphas)
    data['mape'] = np.abs(data['q_0.5']-data['cases'])/data['cases'] 
    data['rwis'] = data['wis']/data['cases']
    data = data[['forecast_date','target_end_date','target','method','wis','mape','FIPS','rwis']]
    data = data.groupby(['forecast_date','target_end_date','target','method']).median().reset_index()
    data = data.rename(columns={'method':'model'})
    data.to_csv(f'Analysis/{fname}_median.csv')

/tmp/ipykernel_474563/2397931273.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby(['forecast_date','target_end_date','target','method']).median().reset_index()
/tmp/ipykernel_474563/2397931273.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby(['forecast_date','target_end_date','target','method']).median().reset_index()
/tmp/ipykernel_474563/2397931273.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns whi